In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
data = pd.read_csv(r"C:\Users\alrem\Desktop\données EURUSD\data_elonmusk.csv", encoding='latin1')

In [3]:
data

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk
...,...,...,...,...,...
3213,Row3213,"@YOUSRC Amos's article was fair, but his edito...",2012-11-20 08:52:03,NaN,elonmusk
3214,Row3214,These articles in Space News describe why Aria...,2012-11-20 08:38:31,NaN,elonmusk
3215,Row3215,Was misquoted by BBC as saying Europe's rocket...,2012-11-20 08:30:44,NaN,elonmusk
3216,Row3216,Just returned from a trip to London and Oxford...,2012-11-19 08:59:46,NaN,elonmusk


In [4]:
tweets = data["Tweet"]

In [5]:
tweets

0       @MeltingIce Assuming max acceleration of 2 to ...
1       RT @SpaceX: BFR is capable of transporting sat...
2                                          @bigajm Yup :)
3                          Part 2 https://t.co/8Fvu57muhM
4       Fly to most places on Earth in under 30 mins a...
                              ...                        
3213    @YOUSRC Amos's article was fair, but his edito...
3214    These articles in Space News describe why Aria...
3215    Was misquoted by BBC as saying Europe's rocket...
3216    Just returned from a trip to London and Oxford...
3217    RT @Jon_Favreau: My Model S just arrived and I...
Name: Tweet, Length: 3218, dtype: object

In [6]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alrem\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
def analyze_sentiment(tweet):
    sentiment = analyzer.polarity_scores(tweet)
    compound_score = sentiment['compound']
    if compound_score >= 0.5:
        return "Positive"
    elif compound_score <= -0.5:
        return "Negative"
    else:
        return "Neutral"

In [8]:
tesla_tweets = data[data['Tweet'].str.lower().str.contains("tesla","TSLA")]

In [9]:
tesla_tweets['Sentiment'] = tesla_tweets['Tweet'].apply(analyze_sentiment)

C:\Users\alrem\AppData\Local\Temp\ipykernel_21480\1332975932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_tweets['Sentiment'] = tesla_tweets['Tweet'].apply(analyze_sentiment)


In [10]:
print(tesla_tweets[['Tweet', 'Sentiment']])

                                                  Tweet Sentiment
28    @Bobby_Gupta Def not ok. Just sent a reminder ...   Neutral
30    @JayCruicks_ATC What if we just made a mini ve...   Neutral
36    Tesla Semi truck unveil & test ride tentativel...   Neutral
41    RT @TeslaMotors: Supercharging Cities https://...   Neutral
45    RT @TeslaMotors: Our new Supercharger route in...   Neutral
...                                                 ...       ...
3188  Tesla was just approved for a full Class 1 Mas...  Positive
3190  Congrats to @dmetcalf for being the first to d...  Positive
3193  Am happy to report that Tesla was narrowly cas...  Positive
3212  Mass. judge denies auto dealers' demand to kil...   Neutral
3217  RT @Jon_Favreau: My Model S just arrived and I...   Neutral

[531 rows x 2 columns]


In [13]:
# Initialisation des variables
current_position = "None"  # Au début, aucune position n'est ouverte
cash_balance = 10000.0  # Solde de trésorerie initial
portfolio = {}  # Dictionnaire pour suivre les positions ouvertes



# Logique de trading
for i in range(len(tesla_tweets)):
    tweet_sentiment = tesla_tweets.iloc[i]['Sentiment'] # Avec la fonction iloc, il est possible de récupérer toutes les données des variables à cette date
    
    # Simulez ici la logique d'achat et de vente en fonction du sentiment
    if tweet_sentiment == "Positive":
        if current_position == "None" and cash_balance > 0:
            # Achat de Tesla
            position_size = 0.2 * cash_balance  # Exemple : investir 20% du solde en TSLA
            cash_balance -= position_size # Cela soustrait le montant de la position achetée (position_size) du solde de trésorerie actuel
            portfolio["TSLA"] = position_size # Cela ajoute la position achetée dans le dictionnaire portfolio en utilisant le symbole de l'action "TSLA" comme clé et la taille de la position (position_size) comme valeur. 
            current_position = "Long"
            print("Achat de Tesla")
    elif tweet_sentiment == "Negative":
        if current_position == "Long" and "TSLA" in portfolio:
            # Vente de Tesla
            position_size = portfolio["TSLA"]
            cash_balance += position_size
            del portfolio["TSLA"]
            current_position = "None"
            print("Vente de Tesla")

# Afficher le solde final du portefeuille
print("Solde final du portefeuille :", cash_balance)

In [12]:
# Importez les bibliothèques nécessaires de QuantConnect
from AlgorithmImports import *
from AlphaModelImports import *

class SentimentBasedTradingAlgorithm(QCAlgorithm):
    def Initialize(self):
        self.SetCash(100000)  # Solde de départ en USD
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 1, 31)
        
        # Ajoutez le symbole de l'action Tesla
        self.tesla = self.AddEquity("TSLA")
        
        # Créez un résolveur de sentiment (à adapter en fonction de votre source de données)
        self.sentiment_resolver = SentimentResolver()
        
        # Planifiez la fonction de trading pour chaque jour de trading
        self.Schedule.On(self.DateRules.EveryDay("TSLA"), self.TimeRules.AfterMarketOpen("TSLA"), self.Trade)

    def Trade(self):
        # Obtenez le sentiment actuel des tweets sur Tesla (à adapter en fonction de votre résolveur de sentiment)
        tweet_sentiment = self.sentiment_resolver.analyze_sentiment()
        
        # Si le sentiment est positif, achetez des actions Tesla
        if tweet_sentiment == "Positive":
            self.SetHoldings("TSLA", 0.5)  # Investissez 50 % du capital dans Tesla
        # Si le sentiment est négatif, vendez des actions Tesla
        elif tweet_sentiment == "Negative":
            self.SetHoldings("TSLA", 0)  # Liquidation de la position
        
        # Affichez le sentiment et les positions
        self.Debug(f"Sentiment: {tweet_sentiment}")
        self.Debug(f"Portfolio Holdings: {self.Portfolio['TSLA'].Quantity}")

# Classe SentimentResolver (à adapter en fonction de votre source de données)
class SentimentResolver:
    def analyze_sentiment(tweet):
        sentiment = analyzer.polarity_scores(tweet)
        compound_score = sentiment['compound']
        if compound_score >= 0.5:
            return "Positive"
        elif compound_score <= -0.5:
            return "Negative"
        else:
            return "Neutral"
        
# Lancez l'algorithme
from AlgorithmImports import *
from AlphaModelImports import *
SentimentBasedTradingAlgorithm()

ModuleNotFoundError: No module named 'AlgorithmImports'